<a href="https://colab.research.google.com/github/sfansaria/CNN_ImageClassification/blob/main/NN_CNN_ImageClassification_on_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, GlobalMaxPooling2D



In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True



In [3]:
#Loading the dataset
 #Splitting between the train and test sets

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(x_train.shape[0],'train_samples')
print(x_test.shape[0],'test_samples')

170498071/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)
50000 train_samples
10000 test_samples


In [4]:
#Coverting the class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [5]:
#Model


model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(80, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.25))

model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes))
model.add(Activation('softmax'))




In [6]:

#Initiate Adam Optimizer

opt = keras.optimizers.Adam(learning_rate=0.0001)

#Train the model
#Implementation of Adam

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 48)        13872     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 48)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 48)        0         
                                                        

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True,write_images=True)

In [8]:
#IMage_Augmentation

if not data_augmentation:
  print('Not using data augmentation')
  model.fit(x_train, y_train, batch_size=batch_size,epochs=epochs,validation_data=(x_test, y_test),shuffle=True,callbacks=[tbCallBack])
else:
  print('Using real-time data augmentation')
  datagen = ImageDataGenerator(
      featurewise_center=False, #set input mean to 0 over the dataset
      samplewise_center=False, #set each sample mean to 0
      featurewise_std_normalization = False,#divide inputs by std of the dataset
      samplewise_std_normalization=False,#divide each imput by its std
      zca_whitening=False,#apply ZCA whitening
      rotation_range=10,#randomly rotate images in the range(degrees-0 to 180)
      width_shift_range=0.2,#randomly shift images horizontally(fraction of total width)
      height_shift_range=0.2,#randomly shift images vertically(fraction of total height)
      horizontal_flip=True, #randomly flip images
      vertical_flip=False)#randomly flip images

      #Compute quantities required for feature-wise normalization
      #std, mean, and principal components if ZCA whitenning is applied
  datagen.fit(x_train)

  #Fit the model on the batches generated by datagen.flow().

  model.fit(datagen.flow(x_train,y_train,batch_size=batch_size), steps_per_epoch = x_train.shape[0]//batch_size,epochs=epochs,validation_data=(x_test, y_test), callbacks=[tbCallBack])



Using real-time data augmentation
Epoch 1/100
1562/1562 [==============================] - 418s 266ms/step - loss: 2.0434 - accuracy: 0.2252 - val_loss: 1.7829 - val_accuracy: 0.3555
Epoch 2/100
1562/1562 [==============================] - 414s 265ms/step - loss: 1.7613 - accuracy: 0.3417 - val_loss: 1.6406 - val_accuracy: 0.4006
Epoch 3/100
1562/1562 [==============================] - 418s 268ms/step - loss: 1.6392 - accuracy: 0.3932 - val_loss: 1.5192 - val_accuracy: 0.4449
Epoch 4/100
1562/1562 [==============================] - 411s 263ms/step - loss: 1.5645 - accuracy: 0.4240 - val_loss: 1.4476 - val_accuracy: 0.4711
Epoch 5/100
1562/1562 [==============================] - 418s 268ms/step - loss: 1.5098 - accuracy: 0.4457 - val_loss: 1.3700 - val_accuracy: 0.5025
Epoch 6/100
1562/1562 [==============================] - 416s 266ms/step - loss: 1.4542 - accuracy: 0.4698 - val_loss: 1.3249 - val_accuracy: 0.5184
Epoch 7/100
1562/1562 [==============================] - 414s 265ms/step

In [9]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("Test Loss",scores[0])
print("Test Accuracy",scores[1])

313/313 [==============================] - 18s 57ms/step - loss: 0.6348 - accuracy: 0.7858
Test Loss 0.6348143815994263
Test Accuracy 0.7857999801635742
